In [2]:
import pandas as pd

### Attribute Information:

1. variance of Wavelet Transformed image (continuous) 
2. skewness of Wavelet Transformed image (continuous) 
3. curtosis of Wavelet Transformed image (continuous) 
4. entropy of image (continuous) 
5. class (integer) 

In [3]:
data = pd.read_csv('datasets/data_banknote_authentication.txt',header=None)

In [4]:
data.columns = ['variance', 'skewness', 'curtosis', 'entropy', 'class']

In [5]:
data['class'].value_counts()

0    762
1    610
Name: class, dtype: int64